# 프롬프트 엔지니어링을 위한 모범 사례

출처: https://help.openai.com/en/articles/6654000-best-practices-for-prompt-engineering-with-openai-api


In [1]:
import re
import requests
import sys
import os
from openai import AzureOpenAI
import tiktoken
from dotenv import load_dotenv
load_dotenv()

client = AzureOpenAI(
  azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT"), 
  api_key=os.getenv("AZURE_OPENAI_KEY"),  
  api_version="2024-02-15-preview"
)

CHAT_COMPLETIONS_MODEL = os.getenv('AZURE_OPENAI_DEPLOYMENT_NAME')

# 1. 최신 모델 사용

최고의 결과를 위해 최신 모델을 사용하세요.

# 2. 프롬프트의 시작 부분에 지침을 배치하고 ### 또는 """로 지침과 컨텍스트를 구분하세요

In [2]:
response = client.chat.completions.create(
        model=CHAT_COMPLETIONS_MODEL,
        messages = [{"role":"system", "content":"당신은 도움이 되는 어시스턴트입니다."},
                                {"role":"user","content": '아래 텍스트를 가장 중요한 요점의 글머리표 목록으로 한글로 요약해 주세요. \n\n \
                ###\n\nWe’re happy to announce that OpenAI and Microsoft are extending our partnership.\
                This multi-year, multi-billion dollar investment from Microsoft follows their previous investments \
                in 2019 and 2021, and will allow us to continue our independent research and develop AI that is \
                increasingly safe, useful, and powerful. \n\n \
                In pursuit of our mission to ensure advanced AI benefits all of humanity, OpenAI remains a \
                capped-profit company and is governed by the OpenAI non-profit. This structure allows us to \
                raise the capital we need to fulfill our mission without sacrificing our core beliefs about \
                broadly sharing benefits and the need to prioritize safety. \
                Microsoft shares this vision and our values, and our partnership is instrumental to our progress. \n###',}],
                max_tokens=400,)

print(response.choices[0].message.content)

- OpenAI와 Microsoft가 파트너십을 확장함을 발표  
- Microsoft가 다년간 수십억 달러 규모의 추가 투자를 진행  
- 이번 투자는 2019년과 2021년에 이어진 투자에 이은 것  
- 투자금을 통해 OpenAI가 독립적인 연구를 계속하고 더 안전하고 유용하며 강력한 AI 개발 가능  
- OpenAI는 제한된 이윤 구조의 기업이며, 비영리 재단(OpenAI 비영리)에 의해 관리됨  
- 이러한 구조는 자본을 조달하면서도 이익 공유와 안전 우선의 가치를 지키는 데 도움  
- Microsoft도 이러한 비전과 가치를 공유하며, 파트너십이 OpenAI 발전에 중요한 역할을 함


# 3. 원하는 컨텍스트, 결과, 길이, 형식, 스타일 등에 대해 구체적이고 상세하게 작성하세요

In [3]:
response = client.chat.completions.create(
    model=CHAT_COMPLETIONS_MODEL,
    messages = [{"role":"system", "content":"You are a helpful assistant."},
                {"role":"user","content": '경봉궁을 위한 시 한편 써줘',}],
        max_tokens=400,)

print(response.choices[0].message.content)


경봉궁에 바치는 시

고요히 빛나는 달빛 아래,  
경봉궁의 옛 담장 따라,  
역사의 숨결이 흐르는 그 곳,  
시간도 잠시 쉬어 가네.

고즈넉한 정원 한 켠에,  
바람이 속삭이는 이야기,  
꽃잎 하나, 나무 한 그루에도  
수백 년의 꿈이 깃들었네.

길 잃은 발걸음 멈추어 서서,  
경봉궁의 아름다움에 취해,  
과거와 현재가 어우러진 공간,  
마음 깊이 스며드는 평화.

경봉궁이여, 그 이름처럼  
세월을 봉인한 고요한 궁전,  
우리의 기억 속에 영원히  
빛나는 별처럼 빛나리라.


In [4]:
response = client.chat.completions.create(
    model=CHAT_COMPLETIONS_MODEL,
    messages = [{"role":"system", "content":"You are a helpful assistant."},
                {"role":"user","content": '경봉궁을 위한 시 한편 써줘 \
                특히 봄에서 여름으로 가는 게절에 아름다운 풍경을 충분히 묘사해줘',}],
        max_tokens=400,)

print(response.choices[0].message.content)



경봉궁 아래 봄에서 여름으로

연둣빛 바람이 살며시 속삭이고  
벚꽃은 졌어도 그 자리에 남아  
푸른 이파리 물결 춤추네.

햇살은 점점 따스해져  
봄의 부드러움에서 여름의 활기로  
경봉궁 연못은 거울처럼 빛나고,  
물결 위에 잔잔한 바람의 노래 흐르네.

꽃들은 이제 다가오는 계절을 맞이해  
진달래 대신 찔레꽃 향기 번지고,  
숲속 나무들은 푸른 옷으로 갈아입으니

새들의 노랫소리로 가득 찬 궁궐,  
그 속에서 나는 조용히 숨을 고른다.  
봄과 여름 사이, 경봉궁의 시간은  
천천히, 그리고 아름답게 흘러가네.


# 4. 원하는 출력 형식을 예제를 통해 명확히 표현하세요 (예제 1, 예제 2).

In [5]:
response = client.chat.completions.create(
    model=CHAT_COMPLETIONS_MODEL,
    messages = [{"role":"system", "content":"You are a helpful assistant."},
                {"role":"user","content": '아래 텍스트에서 회사명 다음 연도를 추출하고 각 엔티티의 시작 인덱스와 끝 인덱스를 출력합니다.\
                Generate output as {"text": "OpenAI", "start": 28, "end": 34} \
                ###\
                OpenAI와 Microsoft가 파트너십을 연장한다는 기쁜 소식을 전하게 되어 기쁩니다. \
                Microsoft의 이번 다년간, 수십억 달러 규모의 투자는 2019년과 2021년에 이루어진 이전 투자에 이은 것입니다. \
                2019년과 2021년 투자에 이은 것으로, 이를 통해 우리는 독립적인 연구를 계속하고 더욱 안전하고 유용하며 강력한 \
                더욱 안전하고 유용하며 강력한 AI를 개발할 수 있게 될 것입니다. \n\n \
                ###\
                ',}],
        max_tokens=400,)

print(response.choices[0].message.content)



[
  {"text": "2019년", "start": 92, "end": 96},
  {"text": "2021년", "start": 99, "end": 103}
]


In [6]:
prompt = """
                아래 텍스트에 언급된 엔티티를 추출합니다. 
                아래 텍스트에 언급된 중요한 엔티티를 추출합니다. 
                먼저 모든 회사 이름을 추출한 다음 모든 연도를 추출합니다, 
                그런 다음 콘텐츠에 맞는 특정 주제를 추출하고 마지막으로 일반적인 주요 주제를 추출합니다.\n\n 
                Desired format: 
                Company names: <comma_separated_list_of_company_names> 
                Years: -||- 
                Specific topics: -||- 
                General themes: -||- 

                ###\n\n
                OpenAI와 Microsoft가 파트너십을 연장한다는 기쁜 소식을 알려드리게 되어 기쁩니다.
                Microsoft의 이번 다년간, 수십억 달러 규모의 투자는 2019년과 2021년 투자에 이어 
                2019년과 2021년 투자에 이은 것으로, 이를 통해 우리는 독립적인 연구를 지속하고 더욱 안전하고 유용하며 강력한 
                더욱 안전하고 유용하며 강력한 AI를 개발할 수 있게 될 것입니다. \n\n

        """

response = client.chat.completions.create(
    model=CHAT_COMPLETIONS_MODEL,
    messages = [{"role":"system", "content":"You are a helpful assistant."},
                {"role":"user","content": prompt}],
        max_tokens=400,)

print(response.choices[0].message.content)



Company names: OpenAI, Microsoft  
Years: 2019, 2021  
Specific topics: 파트너십 연장, 다년간 투자, AI 연구 개발, AI 안전성, AI 유용성, AI 강력화  
General themes: 인공지능, 투자, 연구 협력, 기술 개발


# 5. 제로샷(zero-shot)으로 시작하고, 이후 몇 가지 예제를 제공하세요. 둘 다 효과가 없으면 미세 조정을 시도하세요 (업데이트 필요)

In [7]:
prompt = """
        OpenAI와 Microsoft가 파트너십을 연장한다는 기쁜 소식을 알려드리게 되어 기쁩니다.
        Microsoft의 이번 다년간, 수십억 달러 규모의 투자는 2019년과 2021년 투자에 이어 
        2019년과 2021년 투자에 이은 것으로, 이를 통해 우리는 독립적인 연구를 지속하고 더욱 안전하고 유용하며 강력한 
        더욱 안전하고 유용하며 강력한 AI를 개발할 수 있게 될 것입니다.

"""

response = client.chat.completions.create(
    model=CHAT_COMPLETIONS_MODEL,
    messages = [{"role":"system", "content":"You are a helpful assistant. Extract keywords from the corresponding texts below."},
                {"role":"user","content": prompt,}],
        max_tokens=400,)

print(response.choices[0].message.content)



OpenAI, Microsoft, 파트너십 연장, 다년간 투자, 수십억 달러, 2019년 투자, 2021년 투자, 독립적인 연구, 안전한 AI, 유용한 AI, 강력한 AI, AI 개발


In [8]:
system_prompt = """

        당신은 유용한 조수입니다. 아래 해당 텍스트에서 키워드를 추출하세요..
        텍스트: 스트라이프는 웹 개발자가 웹 사이트와 모바일 애플리케이션에 결제 처리를 통합하는 데 사용할 수 있는 API를 제공합니다. 
        키워드: 스트라이프, 결제 처리, API, 웹 개발자, 웹사이트, 모바일 애플리케이션 
        ###
        텍스트: OpenAI는 텍스트를 이해하고 텍스트를 생성하는 데 매우 능숙한 최첨단 언어 모델을 학습시켰습니다.
        이해하고 텍스트를 생성하는 데 매우 능숙한 최첨단 언어 모델을 학습시켰습니다. API를 통해 이러한 모델에 액세스할 수 있으며, 언어 처리와 관련된 거의 모든 작업을
        언어 처리와 관련된 모든 작업을 해결하는 데 사용할 수 있습니다.
        키워드: 언어 모델, 텍스트 처리, API.
        
"""

user_prompt = """
        텍스트: OpenAI와 Microsoft가 파트너십을 연장한다는 소식을 전하게 되어 기쁘게 생각합니다.
        Microsoft의 이번 다년간, 수십억 달러 규모의 투자는 2019년과 2021년에 이루어진 이전 투자에 이은 것입니다.
        2019년과 2021년 투자에 이은 것으로, 이를 통해 우리는 독립적인 연구를 지속하고 더욱 안전하고 유용하며 강력한 
        더욱 안전하고 유용하며 강력한 AI를 개발할 수 있게 될 것입니다. 
        키워드:
"""

response = client.chat.completions.create(
    model=CHAT_COMPLETIONS_MODEL,
    messages = [{"role":"system", "content": system_prompt},
                {"role":"user","content": user_prompt,}],
        max_tokens=400,)

print(response.choices[0].message.content)



키워드: OpenAI, Microsoft, 파트너십, 다년간 투자, 수십억 달러, 2019년 투자, 2021년 투자, 독립적 연구, 안전한 AI, 유용한 AI, 강력한 AI


# 6. 모호하고 부정확한 설명을 줄이세요

In [9]:
response = client.chat.completions.create(
    model=CHAT_COMPLETIONS_MODEL,
    messages = [{"role":"system", "content":"You are a helpful assistant."},
                {"role":"user","content": '새 제품에 대한 설명을 작성하세요. 이 제품은 차세대 카시트입니다. 이 제품에 대한 설명은 몇 문장으로만 짧게 작성하고 너무 길지 않아야 합니다.',}],
        max_tokens=400,)

print(response.choices[0].message.content)



차세대 카시트는 안전성과 편안함을 한층 강화한 혁신적인 제품입니다. 인체공학적 디자인과 첨단 충격 흡수 기술로 아이의 안전을 최우선으로 고려하였으며, 간편한 설치와 조절 기능으로 사용자 편의성을 높였습니다. 최고의 보호와 편리함을 동시에 제공하는 스마트한 선택입니다.


In [10]:
response = client.chat.completions.create(
    model=CHAT_COMPLETIONS_MODEL,
    messages = [{"role":"system", "content":"You are a helpful assistant."},
                {"role":"user","content": '새 제품에 대한 설명을 작성하세요. 이 제품은 차세대 카시트입니다. 3~5문장으로 구성된 단락을 사용하여 이 제품을 설명하세요.',}],
        max_tokens=400,)

print(response.choices[0].message.content)



이 차세대 카시트는 아이들의 안전과 편안함을 최우선으로 설계되었습니다. 인체공학적 디자인과 최신 충격 흡수 기술을 적용하여 보호 기능을 극대화하였으며, 조절 가능한 기능으로 성장하는 아이들에게 맞춤형 착용감을 제공합니다. 또한, 통기성이 뛰어난 소재를 사용해 장시간 착용에도 쾌적함을 유지시켜 줍니다. 세련된 디자인과 간편한 설치 방법으로 부모님들의 편리함까지 고려한 혁신적인 제품입니다.


# 7. 하지 말아야 할 것을 말하는 대신, 해야 할 것을 명확히 설명하세요

In [11]:
response = client.chat.completions.create(
    model=CHAT_COMPLETIONS_MODEL,
    messages = [{"role":"system", "content":"You are a helpful assistant."},
                {"role":"user","content": 'The following is a conversation between an Agent and a Customer. DO NOT ASK USERNAME OR PASSWORD. DO NOT REPEAT. Customer: I can’t log in to my account.\
                Agent:',}],
        max_tokens=400,)

print(response.choices[0].message.content)


I’m sorry to hear that you're having trouble logging in. Could you please tell me if you’re seeing any specific error messages when you try to log in?


In [12]:
response = client.chat.completions.create(
    model=CHAT_COMPLETIONS_MODEL,
    messages = [{"role":"system", "content":"You are a helpful assistant."},
                {"role":"user","content":'The following is a conversation between an Agent and a Customer. \
                The agent will attempt to diagnose the problem and suggest a solution, whilst refraining from asking any questions related to PII. \
                Instead of asking for PII, such as username or password, refer the user to the help article www.samplewebsite.com/help/faq \n\n\
                Customer: I can’t log in to my account. \n\
                Agent:',}],
        max_tokens=400,)

print(response.choices[0].message.content)


I'm sorry to hear that you're having trouble logging in. Please try clearing your browser's cache and cookies, and ensure that your internet connection is stable. If the problem persists, you can visit our help article for more troubleshooting steps: www.samplewebsite.com/help/faq.


# 8. 코드 생성 관련 - 특정 패턴으로 모델을 유도하기 위해 "선도 단어"를 사용하세요

In [13]:
response = client.chat.completions.create(
    model=CHAT_COMPLETIONS_MODEL,
    messages = [{"role":"system", "content":"You are a helpful assistant."},
                {"role":"user","content":'# 다음과 같은 간단한 파이썬 함수를 작성합니다. \n\
                # 1. 마일 단위로 숫자를 요청하세요.\n\
                # 2. 마일을 킬로미터로 변환합니다.',}],
        max_tokens=400,)

print(response.choices[0].message.content)


아래는 마일 단위의 숫자를 입력받아 이를 킬로미터로 변환하는 간단한 파이썬 함수 예제입니다:

```python
def miles_to_kilometers():
    miles = float(input("마일 단위로 숫자를 입력하세요: "))
    kilometers = miles * 1.60934
    print(f"{miles} 마일은 {kilometers:.2f} 킬로미터입니다.")

# 함수 호출 예시
miles_to_kilometers()
```

- 1마일 = 1.60934킬로미터를 기준으로 변환했습니다.
- 소수점 둘째 자리까지 출력하도록 했습니다.
